In [ ]:
# %pip install vanna


In [1]:
import vanna
from vanna.remote import VannaDefault

In [2]:
api_key = 'aa7f794af5d342d0a13c590204992cd0' # Your API key from https://vanna.ai/account/profile 

vanna_model_name ='yang_test' # Your model name from https://vanna.ai/account/profile 
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [3]:
vn.connect_to_duckdb(url=':memory:')


In [4]:
# running the CSV import command with the name of your file
# After downloading place the file in the same directory as the Python/Jupyter Script
# This will Create a Table with contents of the CSV and select it
vn.run_sql("""CREATE TABLE WARN_test AS SELECT * FROM 'WARN.csv';
SELECT * FROM WARN_test
""")

,State,Company,Address,Number of Workers,Received Date,Effective Date,Layoff/Closure,Temporary/Permanent,Union,Region,County,Industry,Notes
0,California,"Republic National Distributing Company, LLC",5100 Franklin Drive Pleasanton CA 94588,60.0,2025-01-09,2025-03-08,Layoff Permanent,None,None,None,Alameda County,42 Wholesale Trade,None
1,California,"Guitar Center, Inc.",10831 W. Pico Blvd. Los Angeles CA 90064,43.0,2025-01-09,2025-03-10,Closure Permanent,None,None,None,Los Angeles County,44-45 Retail Trade,None
2,California,Qualcomm Incorporated,5775 Morehouse Drive San Diego CA 92121,50.0,2025-01-09,2025-03-10,Layoff Permanent,None,None,None,San Diego County,55 Management of Companies and Enterprises,None
3,California,"Guitar Center, Inc.",6910 Automall Pkwy Gilroy CA 95020,19.0,2025-01-09,2025-03-10,Closure Permanent,None,None,None,Santa Clara County,44-45 Retail Trade,None
4,California,Summit Interconnect,3506 W. Lake Center Drive #A Santa Ana CA 92704,74.0,2025-01-09,2025-03-31,Closure Permanent,None,None,None,Orange County,31-33 Manufacturing,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20394,California,The Boeing Company,Huntington Beach,7.0,2009-01-01,NaT,None,None,None,None,None,None,None
20395,California,The Boeing Company,Huntington Beach,12.0,2009-01-01,NaT,None,None,None,None,None,None,None
20396,California,The Boeing Company,Long Beach,47.0,2009-01-01,NaT,None,None,None,None,None,None,None
20397,California,The Boeing Company,Long Beach,3.0,2009-01-01,NaT,None,None,None,None,None,None,None


In [6]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [7]:
# In duckDB the describe statement can fetch the DDL for any table
vn.train(ddl="DESCRIBE SELECT * FROM WARN_test")

Adding ddl: DESCRIBE SELECT * FROM WARN_test


'465193-ddl'

In [8]:
# here is an example of training on SQL statements
# In the WARN Dataset number of workers are total laid off.
vn.train(
question="Calculate the total number of workers?"
,sql="""SELECT SUM(CAST("Number of Workers" AS INTEGER)) AS Total_Number_of_Workers
FROM WARN_test
""")

'c198c12b80070272539c02059cde9f7e-sql'

In [9]:
# We can use documentation to give explicit context that you would give to a data analyst
vn.train(documentation="The number of worker's column corresponds to people laid off")

Adding documentation....


'2844437-doc'

In [10]:
vn.ask("Which Company did the most amount of layoffs?", visualize=False)


SQL Prompt: [{'role': 'system', 'content': "You are a DuckDB SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nDESCRIBE SELECT * FROM Stackoverflow.users;\n\nDESCRIBE SELECT * FROM listings\n\nDESCRIBE SELECT * FROM WARN_test\n\nDESCRIBE SELECT * FROM WARN_test\n\nDESCRIBE SELECT * FROM WARN_test\n\nDESCRIBE SELECT * FROM WARN_test\n\nDESCRIBE SELECT * FROM WARN_test\n\nDESCRIBE SELECT * FROM table_name\n\n\n    CREATE TABLE IF NOT EXISTS my-table (\n        id INT PRIMARY KEY,\n        name VARCHAR(100),\n        age INT\n    )\n\n\n\n===Additional Context \n\nThe number of worker's column corresponds to people laid off\n\nThe number of worker's column corresponds to people laid off\n\nThe number of worker's column corresponds to people laid off\n\nThe number of worker's column corresponds to people laid off\n\nThe number of worker's c

('SELECT Company, SUM(CAST("Number of Workers" AS INTEGER)) AS Total_Layoffs\nFROM WARN_test\nGROUP BY Company\nORDER BY Total_Layoffs DESC\nLIMIT 1;',
       Company  Total_Layoffs
 0  Tesla, Inc        11083.0,
 None)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None
